## ISI Epilepsy Brno & Montreal Dataset with H2o AutoML, Brno patients

<u>**Goal:**</u> Predict "onset_channel" using H2o AutoML on "outcome=11" & "segm_type=4" for all remaining patients, use "leave one patient out" and summarize the results

7/11/2022

---

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Import dataset

epilepsy = pd.read_csv('/Users/marek/Desktop/Marek/Jobs/ISI/Work/data/new_isi_data.csv')
# Remove unnamed column: ,index_col=0
epilepsy.head()

,Unnamed: 0,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [3]:
# Drop index column

epilepsy = epilepsy.loc[:, ~epilepsy.columns.str.contains('^Unnamed')]
epilepsy.head()

,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,power_4.0-8.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
0,583,ra1,0,NON_SOZ,11.0,11,34,22,0.533375,0.781969,...,0.995615,0.003025,0.985229,0.997413,0.990955,0.004121,0.970733,0.994151,0,0
1,583,ra2,0,NON_SOZ,11.0,16,40,49,0.518898,0.780688,...,0.999169,0.000649,0.996365,0.999503,0.997006,0.001574,0.987965,0.997992,0,0
2,583,ra3,0,NON_SOZ,11.0,20,35,61,0.466505,0.740345,...,0.999172,0.000648,0.996397,0.999509,0.996981,0.001719,0.987949,0.997969,0,0
3,583,ra4,0,NON_SOZ,11.0,27,37,66,0.444008,0.721173,...,0.997592,0.001411,0.991613,0.998744,0.994820,0.002903,0.984822,0.997324,0,0
4,583,ra5,0,NON_SOZ,11.0,11,58,63,0.588096,0.937518,...,0.994437,0.003369,0.981503,0.996723,0.996824,0.001831,0.987556,0.997855,0,0


In [4]:
# Keep only segm_type = 4 

epilepsy_4 = epilepsy.loc[epilepsy['segm_type'] == 4]
epilepsy_4.head()

,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,power_4.0-8.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
34674,583,ra1,0,NON_SOZ,11.0,19,56,50,0.526201,0.474988,...,0.986394,0.008171,0.962278,0.992315,0.951701,0.015386,0.894617,0.968141,4,0
34675,583,ra2,0,NON_SOZ,11.0,35,62,86,0.560877,0.560381,...,0.997199,0.001929,0.989986,0.998410,0.982513,0.007849,0.952962,0.988048,4,0
34676,583,ra3,0,NON_SOZ,11.0,36,66,101,0.558763,0.576171,...,0.997173,0.002038,0.989737,0.998386,0.982065,0.007308,0.951814,0.987657,4,0
34677,583,ra4,0,NON_SOZ,11.0,47,66,102,0.554024,0.565439,...,0.992253,0.005044,0.975965,0.995656,0.975849,0.009962,0.941062,0.984107,4,0
34678,583,ra5,0,NON_SOZ,11.0,32,75,96,0.649021,0.658895,...,0.978247,0.009224,0.944790,0.987954,0.980963,0.007094,0.949647,0.986945,4,0


In [5]:
# Keep only outcome = 11 

epilepsy_11 = epilepsy_4.loc[epilepsy_4['outcome'] == 11]
epilepsy_11.head()

,patient_id,channel_name,resected,onset_channel,outcome,HFO_80.0-250.0,HFO_250.0-600.0,spike_rate,power_1.0-4.0,power_4.0-8.0,...,lin_corr_median_80.0-250.0,ren_80.0-250.0,phase_sync_80.0-250.0,coherence_80.0-250.0,lin_corr_median_250.0-600.0,ren_250.0-600.0,phase_sync_250.0-600.0,coherence_250.0-600.0,segm_type,segm_number
34674,583,ra1,0,NON_SOZ,11.0,19,56,50,0.526201,0.474988,...,0.986394,0.008171,0.962278,0.992315,0.951701,0.015386,0.894617,0.968141,4,0
34675,583,ra2,0,NON_SOZ,11.0,35,62,86,0.560877,0.560381,...,0.997199,0.001929,0.989986,0.998410,0.982513,0.007849,0.952962,0.988048,4,0
34676,583,ra3,0,NON_SOZ,11.0,36,66,101,0.558763,0.576171,...,0.997173,0.002038,0.989737,0.998386,0.982065,0.007308,0.951814,0.987657,4,0
34677,583,ra4,0,NON_SOZ,11.0,47,66,102,0.554024,0.565439,...,0.992253,0.005044,0.975965,0.995656,0.975849,0.009962,0.941062,0.984107,4,0
34678,583,ra5,0,NON_SOZ,11.0,32,75,96,0.649021,0.658895,...,0.978247,0.009224,0.944790,0.987954,0.980963,0.007094,0.949647,0.986945,4,0


In [6]:
# Show unique patient_id

epilepsy_11.patient_id.unique()

array([ 583,  657,  953,  965, 1002, 1021, 1041,  723, 1043,  717, 1630,
         61,   63,   71,   77,   82,   89,   93], dtype=int64)

In [7]:
# Number of unique patients with good outcome = 11 and seqment type = 4

len(epilepsy_11.patient_id.unique())

18

In [8]:
# Onset channels in numbers based on SOZ vs NOZ

epilepsy_11.groupby(['onset_channel', 'patient_id'])['onset_channel'].count()

onset_channel  patient_id
NON_SOZ        61            142
               63            117
               71            160
               77            166
               82             74
               89            161
               93            154
               583            84
               657            37
               717            61
               723            71
               953            81
               965            80
               1002           55
               1021           41
               1041           79
               1043           62
               1630          161
SOZ            61              9
               63              3
               71              9
               77              2
               82             11
               89              9
               93              6
               583             3
               657             8
               717            10
               723            15
               95

In [9]:
# Testing dataframes

testing_df_61 = epilepsy_11.loc[epilepsy_11['patient_id'] == 61]
testing_df_63 = epilepsy_11.loc[epilepsy_11['patient_id'] == 63]
testing_df_71 = epilepsy_11.loc[epilepsy_11['patient_id'] == 71]
testing_df_77 = epilepsy_11.loc[epilepsy_11['patient_id'] == 77]
testing_df_82 = epilepsy_11.loc[epilepsy_11['patient_id'] == 82]
testing_df_89 = epilepsy_11.loc[epilepsy_11['patient_id'] == 89]
testing_df_93 = epilepsy_11.loc[epilepsy_11['patient_id'] == 93]
testing_df_583 = epilepsy_11.loc[epilepsy_11['patient_id'] == 583]
testing_df_657 = epilepsy_11.loc[epilepsy_11['patient_id'] == 657]
testing_df_717 = epilepsy_11.loc[epilepsy_11['patient_id'] == 717]
testing_df_723 = epilepsy_11.loc[epilepsy_11['patient_id'] == 723]
testing_df_953 = epilepsy_11.loc[epilepsy_11['patient_id'] == 953]
testing_df_1002 = epilepsy_11.loc[epilepsy_11['patient_id'] == 1002]
testing_df_1041 = epilepsy_11.loc[epilepsy_11['patient_id'] == 1041]
testing_df_1043 = epilepsy_11.loc[epilepsy_11['patient_id'] == 1043]
testing_df_1630 = epilepsy_11.loc[epilepsy_11['patient_id'] == 1630]

In [10]:
# Testing drop columns no. 0, 1, 2, 4, 65, 66

testing_df_61_drop = testing_df_61.drop(testing_df_61.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_63_drop = testing_df_63.drop(testing_df_63.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_71_drop = testing_df_71.drop(testing_df_71.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_77_drop = testing_df_77.drop(testing_df_77.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_82_drop = testing_df_82.drop(testing_df_82.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_89_drop = testing_df_89.drop(testing_df_89.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_93_drop = testing_df_93.drop(testing_df_93.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_583_drop = testing_df_583.drop(testing_df_583.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_717_drop = testing_df_717.drop(testing_df_717.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_723_drop = testing_df_723.drop(testing_df_723.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_953_drop = testing_df_953.drop(testing_df_953.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_1002_drop = testing_df_1002.drop(testing_df_1002.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_1041_drop = testing_df_1041.drop(testing_df_1041.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
testing_df_1630_drop = testing_df_1630.drop(testing_df_1630.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)

In [11]:
# Training dataframes

delete_row_61 = epilepsy_11[epilepsy_11["patient_id"] == 61].index
training_df_61 = epilepsy_11.drop(delete_row_61)
delete_row_63 = epilepsy_11[epilepsy_11["patient_id"] == 63].index
training_df_63 = epilepsy_11.drop(delete_row_63)
delete_row_71 = epilepsy_11[epilepsy_11["patient_id"] == 71].index
training_df_71 = epilepsy_11.drop(delete_row_71)
delete_row_77 = epilepsy_11[epilepsy_11["patient_id"] == 77].index
training_df_77 = epilepsy_11.drop(delete_row_77)
delete_row_82 = epilepsy_11[epilepsy_11["patient_id"] == 82].index
training_df_82 = epilepsy_11.drop(delete_row_82)
delete_row_89 = epilepsy_11[epilepsy_11["patient_id"] == 89].index
training_df_89 = epilepsy_11.drop(delete_row_89)
delete_row_93 = epilepsy_11[epilepsy_11["patient_id"] == 93].index
training_df_93 = epilepsy_11.drop(delete_row_93)
delete_row_583 = epilepsy_11[epilepsy_11["patient_id"] == 583].index
training_df_583 = epilepsy_11.drop(delete_row_583)
delete_row_717 = epilepsy_11[epilepsy_11["patient_id"] == 717].index
training_df_717 = epilepsy_11.drop(delete_row_717)
delete_row_723 = epilepsy_11[epilepsy_11["patient_id"] == 723].index
training_df_723 = epilepsy_11.drop(delete_row_723)
delete_row_953 = epilepsy_11[epilepsy_11["patient_id"] == 953].index
training_df_953 = epilepsy_11.drop(delete_row_953)
delete_row_1002 = epilepsy_11[epilepsy_11["patient_id"] == 1002].index
training_df_1002 = epilepsy_11.drop(delete_row_1002)
delete_row_1041 = epilepsy_11[epilepsy_11["patient_id"] == 1041].index
training_df_1041 = epilepsy_11.drop(delete_row_1041)
delete_row_1630 = epilepsy_11[epilepsy_11["patient_id"] == 1630].index
training_df_1630 = epilepsy_11.drop(delete_row_1630)

In [12]:
# Training drop columns no. 0, 1, 2, 4, 65, 66

training_df_61_drop = training_df_61.drop(training_df_61.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_63_drop = training_df_63.drop(training_df_63.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_71_drop = training_df_71.drop(training_df_71.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_77_drop = training_df_77.drop(training_df_77.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_82_drop = training_df_82.drop(training_df_82.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_89_drop = training_df_89.drop(training_df_89.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_93_drop = training_df_93.drop(training_df_93.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_583_drop = training_df_583.drop(training_df_583.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_717_drop = training_df_717.drop(training_df_717.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_723_drop = training_df_723.drop(training_df_723.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_953_drop = training_df_953.drop(training_df_953.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_1002_drop = training_df_1002.drop(training_df_1002.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_1041_drop = training_df_1041.drop(training_df_1041.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)
training_df_1630_drop = training_df_1630.drop(training_df_1630.columns[[0, 1, 2, 4, 65, 66 ]], axis=1)

## CLASSIFICATION PROBLEM, Pt 61
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [13]:
# Start H2o

import h2o 
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.333-b02, mixed mode)
  Starting server from C:\Users\marek\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\marek\AppData\Local\Temp\tmpz4jq31od
  JVM stdout: C:\Users\marek\AppData\Local\Temp\tmpz4jq31od\h2o_Rodina_started_from_python.out
  JVM stderr: C:\Users\marek\AppData\Local\Temp\tmpz4jq31od\h2o_Rodina_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,5 months and 11 days !!!
H2O_cluster_name:,H2O_from_python_Rodina_sxpb83
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.531 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [14]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_61_drop)
test = h2o.H2OFrame(testing_df_61)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [15]:
# Define X and y, for binary classification, response should be a factor

X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [16]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [17]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
10:07:26.933: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 55s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_1_20221107_100726

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.005737924995719212
RMSE: 0.07574909237554739
LogLoss: 0.033603115010457874
Null degrees of freedom: 1809
Residual degrees of freedom: 1805
Null deviance: 1109.4669954605124
Residual deviance: 121.64327633785756
AIC: 131.64327633785757
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7475377247448626: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1644.0,0.0,0.0,(0.0/1644.0)
1,SOZ,0.0,166.0,0.0,(0.0/166.0)
2,Total,1644.0,166.0,0.0,(0.0/1810.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.747538,1.000000,91.0
1,max f2,0.747538,1.000000,91.0
2,max f0point5,0.747538,1.000000,91.0
3,max accuracy,0.747538,1.000000,91.0
4,max precision,0.998740,1.000000,0.0
5,max recall,0.747538,1.000000,91.0
6,max specificity,0.998740,1.000000,0.0
7,max absolute_mcc,0.747538,1.000000,91.0
8,max min_per_class_accuracy,0.747538,1.000000,91.0
9,max mean_per_class_accuracy,0.747538,1.000000,91.0



Gains/Lift Table: Avg response rate:  9.17 %, avg score: 11.35 %


...


Gains/Lift Table: Avg response rate:  9.17 %, avg score:  9.16 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010497,0.937405,10.903614,10.903614,1.000000,0.967198,1.000000,0.967198,0.114458,0.114458,990.361446,990.361446,0.114458
1,2,0.020442,0.872357,9.692102,10.314230,0.888889,0.908773,0.945946,0.938775,0.096386,0.210843,869.210174,931.422989,0.209627
2,3,0.030387,0.780518,7.874833,9.515882,0.722222,0.825373,0.872727,0.901662,0.078313,0.289157,687.483266,851.588171,0.284899
3,4,0.040331,0.681233,9.086345,9.409969,0.833333,0.731487,0.863014,0.859701,0.090361,0.379518,808.634538,840.996864,0.373435
4,5,0.050276,0.602897,7.269076,8.986495,0.666667,0.635546,0.824176,0.815362,0.072289,0.451807,626.907631,798.649543,0.442075
5,6,0.100000,0.284128,4.724900,6.867470,0.433333,0.445493,0.629834,0.631449,0.234940,0.686747,372.489960,586.746988,0.645993
6,7,0.150276,0.145653,2.755859,5.491894,0.252747,0.207586,0.503676,0.489642,0.138554,0.825301,175.585860,449.189405,0.743184
7,8,0.200000,0.090049,1.090361,4.397590,0.100000,0.114787,0.403315,0.396446,0.054217,0.879518,9.036145,339.759036,0.748131
8,9,0.300000,0.039577,0.602410,3.132530,0.055249,0.060089,0.287293,0.284327,0.060241,0.939759,-39.759036,213.253012,0.704358
9,10,0.400000,0.020388,0.180723,2.394578,0.016575,0.029159,0.219613,0.220535,0.018072,0.957831,-81.927711,139.457831,0.614157




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.943109,0.009700,0.935135,0.945087,0.954424,0.949580,0.931319
1,auc,0.938615,0.016126,0.922157,0.929204,0.945647,0.963099,0.932970
2,err,0.056891,0.009700,0.064865,0.054913,0.045576,0.050420,0.068681
3,err_count,20.600000,3.646916,24.000000,19.000000,17.000000,18.000000,25.000000
4,f0point5,0.692224,0.069642,0.602410,0.718391,0.791367,0.684211,0.664740
5,f1,0.692339,0.052310,0.625000,0.724638,0.721311,0.742857,0.647887
6,f2,0.697473,0.074477,0.649351,0.730994,0.662651,0.812500,0.631868
7,lift_top_group,10.980957,1.087697,12.333333,10.176471,10.657143,11.900000,9.837838
8,logloss,0.153367,0.022178,0.168932,0.160964,0.145054,0.118664,0.173220
9,max_per_class_error,0.296236,0.101623,0.333333,0.264706,0.371429,0.133333,0.378378



See the whole table with table.as_data_frame()


In [18]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_1_20221107_100726,0.938821,0.153436,0.722644,0.188689,0.208992,0.0436775
StackedEnsemble_AllModels_3_AutoML_1_20221107_100726,0.938731,0.155476,0.714237,0.139122,0.211779,0.0448502
StackedEnsemble_AllModels_4_AutoML_1_20221107_100726,0.936518,0.155235,0.720299,0.193526,0.209745,0.0439928


In [19]:
# leader's model 

model = h2o.get_model('StackedEnsemble_BestOfFamily_4_AutoML_1_20221107_100726')

In [20]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.05196281329257362
RMSE: 0.2279535331873003
LogLoss: 0.20294886643973073
Null degrees of freedom: 150
Residual degrees of freedom: 146
Null deviance: 70.32294986108386
Residual deviance: 61.29055766479869
AIC: 71.29055766479868
AUC: 0.7809076682316118
AUCPR: 0.19355297320035425
Gini: 0.5618153364632237

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21654129862521246: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,138.0,4.0,0.0282,(4.0/142.0)
1,SOZ,6.0,3.0,0.6667,(6.0/9.0)
2,Total,144.0,7.0,0.0662,(10.0/151.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.216541,0.375000,6.0
1,max f2,0.090498,0.454545,29.0
2,max f0point5,0.216541,0.405405,6.0
3,max accuracy,0.308282,0.933775,0.0
4,max precision,0.216541,0.428571,6.0
5,max recall,0.009831,1.000000,90.0
6,max specificity,0.308282,0.992958,0.0
7,max absolute_mcc,0.216541,0.343612,6.0
8,max min_per_class_accuracy,0.090498,0.666667,29.0
9,max mean_per_class_accuracy,0.090498,0.748826,29.0



Gains/Lift Table: Avg response rate:  5.96 %, avg score:  5.01 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.013245,0.272297,0.000000,0.000000,0.000000,0.291925,0.000000,0.291925,0.000000,0.000000,-100.000000,-100.000000,-0.014085
1,2,0.026490,0.264994,8.388889,4.194444,0.500000,0.267010,0.250000,0.279467,0.111111,0.111111,738.888889,319.444444,0.089984
2,3,0.033113,0.233387,16.777778,6.711111,1.000000,0.233481,0.400000,0.270270,0.111111,0.222222,1577.777778,571.111111,0.201095
3,4,0.046358,0.216541,8.388889,7.190476,0.500000,0.224917,0.428571,0.257312,0.111111,0.333333,738.888889,619.047619,0.305164
4,5,0.052980,0.206284,0.000000,6.291667,0.000000,0.210884,0.375000,0.251509,0.000000,0.333333,-100.000000,529.166667,0.298122
5,6,0.105960,0.140236,0.000000,3.145833,0.000000,0.169572,0.187500,0.210541,0.000000,0.333333,-100.000000,214.583333,0.241784
6,7,0.152318,0.112442,4.793651,3.647343,0.285714,0.127659,0.217391,0.185316,0.222222,0.555556,379.365079,264.734300,0.428795
7,8,0.205298,0.089561,2.097222,3.247312,0.125000,0.098120,0.193548,0.162813,0.111111,0.666667,109.722222,224.731183,0.490610
8,9,0.304636,0.058269,0.000000,2.188406,0.000000,0.074681,0.130435,0.134075,0.000000,0.666667,-100.000000,118.840580,0.384977
9,10,0.403974,0.032640,0.000000,1.650273,0.000000,0.043908,0.098361,0.111902,0.000000,0.666667,-100.000000,65.027322,0.279343


## CLASSIFICATION PROBLEM, Pt 63
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [22]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_63_drop)
test = h2o.H2OFrame(testing_df_63)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
# Define X and y, for binary classification, response should be a factor

X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [24]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [25]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
10:30:02.753: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 55s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_2_20221107_103002

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0058986730273898535
RMSE: 0.07680281913699427
LogLoss: 0.03606633769345347
Null degrees of freedom: 1840
Residual degrees of freedom: 1836
Null deviance: 1142.8812300931922
Residual deviance: 132.7962553872956
AIC: 142.7962553872956
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7400985058232222: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1669.0,0.0,0.0,(0.0/1669.0)
1,SOZ,0.0,172.0,0.0,(0.0/172.0)
2,Total,1669.0,172.0,0.0,(0.0/1841.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.740099,1.000000,96.0
1,max f2,0.740099,1.000000,96.0
2,max f0point5,0.740099,1.000000,96.0
3,max accuracy,0.740099,1.000000,96.0
4,max precision,0.999464,1.000000,0.0
5,max recall,0.740099,1.000000,96.0
6,max specificity,0.999464,1.000000,0.0
7,max absolute_mcc,0.740099,1.000000,96.0
8,max min_per_class_accuracy,0.740099,1.000000,96.0
9,max mean_per_class_accuracy,0.740099,1.000000,96.0



Gains/Lift Table: Avg response rate:  9.34 %, avg score: 11.63 %


...


Gains/Lift Table: Avg response rate:  9.34 %, avg score:  9.36 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010320,0.940269,10.703488,10.703488,1.000000,0.974076,1.000000,0.974076,0.110465,0.110465,970.348837,970.348837,0.110465
1,2,0.020098,0.868058,10.108850,10.414205,0.944444,0.904033,0.972973,0.940001,0.098837,0.209302,910.885013,941.420490,0.208703
2,3,0.030418,0.788678,5.633415,8.792151,0.526316,0.833791,0.821429,0.903965,0.058140,0.267442,463.341493,779.215116,0.261450
3,4,0.040196,0.662705,7.730297,8.533862,0.722222,0.706651,0.797297,0.855970,0.075581,0.343023,673.029716,753.386235,0.334036
4,5,0.050516,0.606891,7.323439,8.286572,0.684211,0.635176,0.774194,0.810861,0.075581,0.418605,632.343941,728.657164,0.406022
5,6,0.100489,0.309792,4.653691,6.479950,0.434783,0.435074,0.605405,0.623983,0.232558,0.651163,365.369060,547.994972,0.607424
6,7,0.150462,0.175349,3.141241,5.371065,0.293478,0.228916,0.501805,0.492770,0.156977,0.808140,214.124115,437.106456,0.725455
7,8,0.200435,0.096390,1.396107,4.380018,0.130435,0.132211,0.409214,0.402874,0.069767,0.877907,39.610718,338.001828,0.747290
8,9,0.300380,0.044045,0.698054,3.154916,0.065217,0.065410,0.294756,0.290590,0.069767,0.947674,-30.194641,215.491610,0.714002
9,10,0.400326,0.022076,0.116342,2.396303,0.010870,0.031403,0.223881,0.225881,0.011628,0.959302,-88.365774,139.630337,0.616582




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.937208,0.018229,0.922872,0.936963,0.916010,0.950954,0.959239
1,auc,0.940871,0.021839,0.919942,0.955696,0.914595,0.961261,0.952860
2,err,0.062792,0.018229,0.077128,0.063037,0.083989,0.049046,0.040761
3,err_count,23.200000,7.190271,29.000000,22.000000,32.000000,18.000000,15.000000
4,f0point5,0.672271,0.101798,0.537634,0.658537,0.620000,0.784314,0.760870
5,f1,0.682785,0.064873,0.579710,0.710526,0.659574,0.727273,0.736842
6,f2,0.699431,0.052109,0.628931,0.771429,0.704545,0.677966,0.714286
7,lift_top_group,10.873221,1.495267,12.533334,10.575758,9.071428,9.918919,12.266666
8,logloss,0.158707,0.032316,0.177083,0.140494,0.206435,0.141200,0.128321
9,max_per_class_error,0.285682,0.067345,0.333333,0.181818,0.261905,0.351351,0.300000



See the whole table with table.as_data_frame()


In [26]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_2_20221107_103002,0.939124,0.159321,0.705016,0.203333,0.215117,0.0462753
StackedEnsemble_AllModels_4_AutoML_2_20221107_103002,0.938037,0.158911,0.709752,0.151752,0.214526,0.0460212
StackedEnsemble_AllModels_3_AutoML_2_20221107_103002,0.937825,0.159793,0.702635,0.175939,0.215968,0.0466422


In [27]:
# leader's model 

model = h2o.get_model('StackedEnsemble_BestOfFamily_4_AutoML_2_20221107_103002')

In [28]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.02418300863246662
RMSE: 0.15550886994787988
LogLoss: 0.0974362973757937
Null degrees of freedom: 119
Residual degrees of freedom: 115
Null deviance: 37.17513449818862
Residual deviance: 23.384711370190498
AIC: 33.3847113701905
AUC: 0.8603988603988605
AUCPR: 0.10247011805267042
Gini: 0.720797720797721

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.11330890533132615: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,110.0,7.0,0.0598,(7.0/117.0)
1,SOZ,1.0,2.0,0.3333,(1.0/3.0)
2,Total,111.0,9.0,0.0667,(8.0/120.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.113309,0.333333,8.0
1,max f2,0.113309,0.476190,8.0
2,max f0point5,0.113309,0.256410,8.0
3,max accuracy,0.372192,0.966667,0.0
4,max precision,0.113309,0.222222,8.0
5,max recall,0.026555,1.000000,37.0
6,max specificity,0.372192,0.991453,0.0
7,max absolute_mcc,0.113309,0.359702,8.0
8,max min_per_class_accuracy,0.026555,0.700855,37.0
9,max mean_per_class_accuracy,0.026555,0.850427,37.0



Gains/Lift Table: Avg response rate:  2.50 %, avg score:  3.20 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.016667,0.200592,0.000000,0.000000,0.000000,0.286729,0.000000,0.286729,0.000000,0.000000,-100.000000,-100.000000,-0.017094
1,2,0.025000,0.195693,0.000000,0.000000,0.000000,0.197718,0.000000,0.257059,0.000000,0.000000,-100.000000,-100.000000,-0.025641
2,3,0.033333,0.163200,0.000000,0.000000,0.000000,0.192389,0.000000,0.240891,0.000000,0.000000,-100.000000,-100.000000,-0.034188
3,4,0.041667,0.137809,0.000000,0.000000,0.000000,0.141180,0.000000,0.220949,0.000000,0.000000,-100.000000,-100.000000,-0.042735
4,5,0.050000,0.131307,0.000000,0.000000,0.000000,0.136745,0.000000,0.206915,0.000000,0.000000,-100.000000,-100.000000,-0.051282
5,6,0.100000,0.075606,13.333333,6.666667,0.333333,0.108930,0.166667,0.157923,0.666667,0.666667,1233.333333,566.666667,0.581197
6,7,0.150000,0.050494,0.000000,4.444444,0.000000,0.064948,0.111111,0.126931,0.000000,0.666667,-100.000000,344.444444,0.529915
7,8,0.200000,0.042679,0.000000,3.333333,0.000000,0.046125,0.083333,0.106730,0.000000,0.666667,-100.000000,233.333333,0.478632
8,9,0.300000,0.031041,0.000000,2.222222,0.000000,0.035615,0.055556,0.083025,0.000000,0.666667,-100.000000,122.222222,0.376068
9,10,0.400000,0.019105,3.333333,2.500000,0.083333,0.023420,0.062500,0.068123,0.333333,1.000000,233.333333,150.000000,0.615385


## CLASSIFICATION PROBLEM, Pt 71
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [29]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_71_drop)
test = h2o.H2OFrame(testing_df_71)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [30]:
# Define X and y, for binary classification, response should be a factor

X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [31]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [32]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
10:55:00.56: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 53s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_3_20221107_105500

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0029340020386705414
RMSE: 0.054166429074386485
LogLoss: 0.026518774578664472
Null degrees of freedom: 1791
Residual degrees of freedom: 1784
Null deviance: 1105.9857900642037
Residual deviance: 95.04328808993348
AIC: 111.04328808993348
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8438634721078595: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1626.0,0.0,0.0,(0.0/1626.0)
1,SOZ,0.0,166.0,0.0,(0.0/166.0)
2,Total,1626.0,166.0,0.0,(0.0/1792.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.843863,1.000000,95.0
1,max f2,0.843863,1.000000,95.0
2,max f0point5,0.843863,1.000000,95.0
3,max accuracy,0.843863,1.000000,95.0
4,max precision,0.998693,1.000000,0.0
5,max recall,0.843863,1.000000,95.0
6,max specificity,0.998693,1.000000,0.0
7,max absolute_mcc,0.843863,1.000000,95.0
8,max min_per_class_accuracy,0.843863,1.000000,95.0
9,max mean_per_class_accuracy,0.843863,1.000000,95.0



Gains/Lift Table: Avg response rate:  9.26 %, avg score: 11.11 %


...


Gains/Lift Table: Avg response rate:  9.26 %, avg score:  9.25 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010045,0.947794,10.795181,10.795181,1.000000,0.970541,1.000000,0.970541,0.108434,0.108434,979.518072,979.518072,0.108434
1,2,0.020089,0.883026,10.795181,10.795181,1.000000,0.918470,1.000000,0.944506,0.108434,0.216867,979.518072,979.518072,0.216867
2,3,0.030134,0.806235,6.597055,9.395805,0.611111,0.842393,0.870370,0.910468,0.066265,0.283133,559.705489,839.580544,0.278827
3,4,0.040179,0.712228,7.196787,8.846051,0.666667,0.767575,0.819444,0.874745,0.072289,0.355422,619.678715,784.605087,0.347427
4,5,0.050223,0.594376,6.597055,8.396252,0.611111,0.672670,0.777778,0.834330,0.066265,0.421687,559.705489,739.625167,0.409387
5,6,0.100446,0.295614,4.797858,6.597055,0.444444,0.431434,0.611111,0.632882,0.240964,0.662651,379.785810,559.705489,0.619600
6,7,0.150112,0.145206,2.425883,5.217002,0.224719,0.206162,0.483271,0.491699,0.120482,0.783133,142.588331,421.700184,0.697647
7,8,0.200335,0.085319,1.919143,4.390241,0.177778,0.109514,0.406685,0.395887,0.096386,0.879518,91.914324,339.024063,0.748522
8,9,0.300223,0.039317,0.603083,3.130201,0.055866,0.057384,0.289963,0.283262,0.060241,0.939759,-39.691728,213.020110,0.704827
9,10,0.400112,0.023502,0.180925,2.393910,0.016760,0.030788,0.221757,0.220232,0.018072,0.957831,-81.907518,139.391037,0.614658




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.934815,0.007618,0.923913,0.938596,0.937669,0.943343,0.930556
1,auc,0.939358,0.021762,0.939496,0.948893,0.950104,0.956347,0.901951
2,err,0.065185,0.007618,0.076087,0.061404,0.062331,0.056657,0.069444
3,err_count,23.400000,3.209361,28.000000,21.000000,23.000000,20.000000,25.000000
4,f0point5,0.647091,0.077262,0.539216,0.714286,0.714286,0.671642,0.596027
5,f1,0.653839,0.058009,0.611111,0.695652,0.729412,0.642857,0.590164
6,f2,0.665828,0.065303,0.705128,0.677966,0.745192,0.616438,0.584416
7,lift_top_group,11.004485,1.717932,13.142858,9.500000,9.000000,11.766666,11.612904
8,logloss,0.159436,0.015526,0.149123,0.161910,0.160835,0.142296,0.183015
9,max_per_class_error,0.322175,0.091367,0.214286,0.333333,0.243902,0.400000,0.419355



See the whole table with table.as_data_frame()


In [33]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_3_20221107_105500,0.938285,0.159439,0.705194,0.184483,0.216258,0.0467677
StackedEnsemble_AllModels_4_AutoML_3_20221107_105500,0.937332,0.160035,0.706717,0.22886,0.216004,0.0466576
StackedEnsemble_Best1000_1_AutoML_3_20221107_105500,0.937023,0.158928,0.710295,0.162358,0.214977,0.0462152


In [34]:
# leader's model 

model = h2o.get_model('StackedEnsemble_AllModels_3_AutoML_3_20221107_105500')

In [35]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.04540626264500404
RMSE: 0.21308745304452828
LogLoss: 0.16257441513605272
Null degrees of freedom: 168
Residual degrees of freedom: 161
Null deviance: 73.9307735898886
Residual deviance: 54.9501523159858
AIC: 70.9501523159858
AUC: 0.8854166666666666
AUCPR: 0.2412595687333811
Gini: 0.7708333333333333

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21772320635026904: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,152.0,8.0,0.05,(8.0/160.0)
1,SOZ,4.0,5.0,0.4444,(4.0/9.0)
2,Total,156.0,13.0,0.071,(12.0/169.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.217723,0.454545,12.0
1,max f2,0.090329,0.540541,37.0
2,max f0point5,0.217723,0.409836,12.0
3,max accuracy,0.675154,0.940828,0.0
4,max precision,0.217723,0.384615,12.0
5,max recall,0.049000,1.000000,71.0
6,max specificity,0.675154,0.993750,0.0
7,max absolute_mcc,0.217723,0.426006,12.0
8,max min_per_class_accuracy,0.090329,0.812500,37.0
9,max mean_per_class_accuracy,0.090329,0.850694,37.0



Gains/Lift Table: Avg response rate:  5.33 %, avg score:  7.11 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011834,0.549195,0.000000,0.000000,0.000000,0.638283,0.000000,0.638283,0.000000,0.000000,-100.000000,-100.000000,-0.012500
1,2,0.023669,0.326076,9.388889,4.694444,0.500000,0.428649,0.250000,0.533466,0.111111,0.111111,838.888889,369.444444,0.092361
2,3,0.035503,0.297171,0.000000,3.129630,0.000000,0.305850,0.166667,0.457594,0.000000,0.111111,-100.000000,212.962963,0.079861
3,4,0.041420,0.276532,18.777778,5.365079,1.000000,0.292696,0.285714,0.434037,0.111111,0.222222,1777.777778,436.507937,0.190972
4,5,0.053254,0.245053,9.388889,6.259259,0.500000,0.260571,0.333333,0.395489,0.111111,0.333333,838.888889,525.925926,0.295833
5,6,0.100592,0.159630,4.694444,5.522876,0.250000,0.206668,0.294118,0.306632,0.222222,0.555556,369.444444,452.287582,0.480556
6,7,0.153846,0.111265,2.086420,4.333333,0.111111,0.136069,0.230769,0.247591,0.111111,0.666667,108.641975,333.333333,0.541667
7,8,0.201183,0.097810,2.347222,3.866013,0.125000,0.105981,0.205882,0.214271,0.111111,0.777778,134.722222,286.601307,0.609028
8,9,0.301775,0.074166,1.104575,2.945534,0.058824,0.084614,0.156863,0.171052,0.111111,0.888889,10.457516,194.553377,0.620139
9,10,0.402367,0.052521,0.000000,2.209150,0.000000,0.064375,0.117647,0.144383,0.000000,0.888889,-100.000000,120.915033,0.513889


## CLASSIFICATION PROBLEM, Pt 77
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [36]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_77_drop)
test = h2o.H2OFrame(testing_df_77)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
# Define X and y, for binary classification, response should be a factor

X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [38]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [39]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
11:05:59.825: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████████
11:15:30.277: GBM_lr_annealing_selection_AutoML_4_20221107_110559 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

███| (done) 100%
Wall time: 9min 54s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_4_20221107_110559

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0034377713771249568
RMSE: 0.058632511264015946
LogLoss: 0.029070765087922498
Null degrees of freedom: 1792
Residual degrees of freedom: 1782
Null deviance: 1137.8139491981344
Residual deviance: 104.24776360529007
AIC: 126.24776360529007
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7984819310663787: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1620.0,0.0,0.0,(0.0/1620.0)
1,SOZ,0.0,173.0,0.0,(0.0/173.0)
2,Total,1620.0,173.0,0.0,(0.0/1793.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.798482,1.000000,101.0
1,max f2,0.798482,1.000000,101.0
2,max f0point5,0.798482,1.000000,101.0
3,max accuracy,0.798482,1.000000,101.0
4,max precision,0.998783,1.000000,0.0
5,max recall,0.798482,1.000000,101.0
6,max specificity,0.998783,1.000000,0.0
7,max absolute_mcc,0.798482,1.000000,101.0
8,max min_per_class_accuracy,0.798482,1.000000,101.0
9,max mean_per_class_accuracy,0.798482,1.000000,101.0



Gains/Lift Table: Avg response rate:  9.65 %, avg score: 11.65 %


...


Gains/Lift Table: Avg response rate:  9.65 %, avg score:  9.62 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010039,0.956746,10.364162,10.364162,1.000000,0.978784,1.000000,0.978784,0.104046,0.104046,936.416185,936.416185,0.104046
1,2,0.020078,0.879001,9.788375,10.076268,0.944444,0.927812,0.972222,0.953298,0.098266,0.202312,878.837508,907.626846,0.201695
2,3,0.030117,0.810097,7.485228,9.212588,0.722222,0.848257,0.888889,0.918284,0.075145,0.277457,648.522800,821.258831,0.273753
3,4,0.040156,0.746489,6.909441,8.636802,0.666667,0.784658,0.833333,0.884878,0.069364,0.346821,590.944123,763.680154,0.339413
4,5,0.050195,0.658124,6.909441,8.291329,0.666667,0.702104,0.800000,0.848323,0.069364,0.416185,590.944123,729.132948,0.405074
5,6,0.100390,0.302888,4.606294,6.448812,0.444444,0.456523,0.622222,0.652423,0.231214,0.647399,360.629416,544.881182,0.605424
6,7,0.150028,0.148327,2.911281,5.278402,0.280899,0.216440,0.509294,0.508176,0.144509,0.791908,191.128142,427.840213,0.710426
7,8,0.200223,0.089815,1.497046,4.330430,0.144444,0.115559,0.417827,0.409748,0.075145,0.867052,49.704560,333.042974,0.738040
8,9,0.300056,0.041577,0.810605,3.159336,0.078212,0.061251,0.304833,0.293798,0.080925,0.947977,-18.939516,215.933558,0.717113
9,10,0.399888,0.024802,0.173701,2.413968,0.016760,0.032712,0.232915,0.228618,0.017341,0.965318,-82.629896,141.396796,0.625812




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.929615,0.018488,0.929348,0.912023,0.935310,0.957507,0.913889
1,auc,0.936180,0.009646,0.948884,0.936041,0.939308,0.922095,0.934575
2,err,0.070385,0.018488,0.070652,0.087977,0.064690,0.042493,0.086111
3,err_count,25.200000,6.379655,26.000000,30.000000,24.000000,15.000000,31.000000
4,f0point5,0.646607,0.077841,0.653153,0.588235,0.684211,0.751880,0.555556
5,f1,0.677182,0.040105,0.690476,0.666667,0.684211,0.727273,0.617284
6,f2,0.716887,0.034314,0.732323,0.769231,0.684211,0.704225,0.694444
7,lift_top_group,10.454346,1.088073,9.684211,9.742857,9.763158,12.172414,10.909091
8,logloss,0.164733,0.012686,0.164518,0.176687,0.160761,0.145769,0.175930
9,max_per_class_error,0.249652,0.070141,0.236842,0.142857,0.315789,0.310345,0.242424



See the whole table with table.as_data_frame()


In [40]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_4_20221107_110559,0.936288,0.164655,0.708973,0.173942,0.219534,0.0481952
StackedEnsemble_AllModels_3_AutoML_4_20221107_110559,0.935726,0.163717,0.713354,0.158144,0.218146,0.0475878
StackedEnsemble_BestOfFamily_6_AutoML_4_20221107_110559,0.93569,0.164604,0.708068,0.136538,0.218968,0.0479472


In [42]:
# leader's model 

model = h2o.get_model('StackedEnsemble_AllModels_4_AutoML_4_20221107_110559')

In [43]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.013731075535465685
RMSE: 0.11717967202320412
LogLoss: 0.058415820384847675
Null degrees of freedom: 167
Residual degrees of freedom: 157
Null deviance: 43.03947858215891
Residual deviance: 19.62771564930882
AIC: 41.62771564930882
AUC: 0.9909638554216867
AUCPR: 0.6652846730286852
Gini: 0.9819277108433735

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7173011105660244: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,166.0,0.0,0.0,(0.0/166.0)
1,SOZ,1.0,1.0,0.5,(1.0/2.0)
2,Total,167.0,1.0,0.006,(1.0/168.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.717301,0.666667,0.0
1,max f2,0.508794,0.769231,4.0
2,max f0point5,0.717301,0.833333,0.0
3,max accuracy,0.717301,0.994048,0.0
4,max precision,0.717301,1.000000,0.0
5,max recall,0.508794,1.000000,4.0
6,max specificity,0.717301,1.000000,0.0
7,max absolute_mcc,0.717301,0.704987,0.0
8,max min_per_class_accuracy,0.508794,0.981928,4.0
9,max mean_per_class_accuracy,0.508794,0.990964,4.0



Gains/Lift Table: Avg response rate:  1.19 %, avg score:  5.09 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011905,0.595105,42.0,42.000000,0.5,0.668214,0.500000,0.668214,0.5,0.5,4100.0,4100.000000,0.493976
1,2,0.023810,0.514276,0.0,21.000000,0.0,0.550187,0.250000,0.609200,0.0,0.5,-100.0,2000.000000,0.481928
2,3,0.035714,0.396850,42.0,28.000000,0.5,0.453028,0.333333,0.557143,0.5,1.0,4100.0,2700.000000,0.975904
3,4,0.041667,0.350195,0.0,24.000000,0.0,0.356018,0.285714,0.528411,0.0,1.0,-100.0,2300.000000,0.969880
4,5,0.053571,0.335175,0.0,18.666667,0.0,0.342076,0.222222,0.487003,0.0,1.0,-100.0,1766.666667,0.957831
5,6,0.101190,0.077899,0.0,9.882353,0.0,0.207037,0.117647,0.355254,0.0,1.0,-100.0,888.235294,0.909639
6,7,0.154762,0.035296,0.0,6.461538,0.0,0.050619,0.076923,0.249804,0.0,1.0,-100.0,546.153846,0.855422
7,8,0.202381,0.029141,0.0,4.941176,0.0,0.031607,0.058824,0.198463,0.0,1.0,-100.0,394.117647,0.807229
8,9,0.303571,0.024318,0.0,3.294118,0.0,0.027348,0.039216,0.141425,0.0,1.0,-100.0,229.411765,0.704819
9,10,0.398810,0.019699,0.0,2.507463,0.0,0.022056,0.029851,0.112919,0.0,1.0,-100.0,150.746269,0.608434


## CLASSIFICATION PROBLEM, Pt 82
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [44]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_82_drop)
test = h2o.H2OFrame(testing_df_82)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [45]:
X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [46]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [47]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
11:18:13.869: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 58s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_5_20221107_111813

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.005783618526780366
RMSE: 0.07605010536994913
LogLoss: 0.03279651766831246
Null degrees of freedom: 1875
Residual degrees of freedom: 1871
Null deviance: 1112.5721276593922
Residual deviance: 123.0525342915083
AIC: 133.0525342915083
AUC: 0.9999928766811033
AUCPR: 0.9999256385786963
Gini: 0.9999857533622065

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.662686420090233: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1711.0,1.0,0.0006,(1.0/1712.0)
1,SOZ,0.0,164.0,0.0,(0.0/164.0)
2,Total,1711.0,165.0,0.0005,(1.0/1876.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.662686,0.996960,85.0
1,max f2,0.662686,0.998782,85.0
2,max f0point5,0.715345,0.997537,82.0
3,max accuracy,0.662686,0.999467,85.0
4,max precision,0.999011,1.000000,0.0
5,max recall,0.662686,1.000000,85.0
6,max specificity,0.999011,1.000000,0.0
7,max absolute_mcc,0.662686,0.996674,85.0
8,max min_per_class_accuracy,0.662686,0.999416,85.0
9,max mean_per_class_accuracy,0.662686,0.999708,85.0



Gains/Lift Table: Avg response rate:  8.74 %, avg score: 10.73 %


...


Gains/Lift Table: Avg response rate:  8.74 %, avg score:  8.75 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010128,0.903924,10.836970,10.836970,0.947368,0.949645,0.947368,0.949645,0.109756,0.109756,983.697047,983.697047,0.109172
1,2,0.020256,0.835478,9.632863,10.234917,0.842105,0.873399,0.894737,0.911522,0.097561,0.207317,863.286264,923.491656,0.204981
2,3,0.030384,0.739738,7.826701,9.432178,0.684211,0.783998,0.824561,0.869014,0.079268,0.286585,682.670090,843.217801,0.280744
3,4,0.040512,0.662888,7.826701,9.030809,0.684211,0.695330,0.789474,0.825593,0.079268,0.365854,682.670090,803.080873,0.356508
4,5,0.050107,0.605936,8.261518,8.883498,0.722222,0.639543,0.776596,0.789967,0.079268,0.445122,726.151762,788.349766,0.432856
5,6,0.100213,0.275965,5.111053,6.997276,0.446809,0.430081,0.611702,0.610024,0.256098,0.701220,411.105345,599.727556,0.658579
6,7,0.150320,0.135039,2.433835,5.476129,0.212766,0.194731,0.478723,0.471593,0.121951,0.823171,143.383498,447.612870,0.737306
7,8,0.200426,0.082417,1.825376,4.563441,0.159574,0.104460,0.398936,0.379810,0.091463,0.914634,82.537623,356.344058,0.782625
8,9,0.300107,0.036897,0.305856,3.149287,0.026738,0.055181,0.275311,0.271985,0.030488,0.945122,-69.414373,214.928735,0.706804
9,10,0.400320,0.020553,0.243383,2.421844,0.021277,0.028029,0.211718,0.210914,0.024390,0.969512,-75.661650,142.184405,0.623718




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.940152,0.011415,0.942857,0.927374,0.940104,0.932976,0.957447
1,auc,0.944657,0.008552,0.945079,0.936488,0.938774,0.944465,0.958478
2,err,0.059848,0.011415,0.057143,0.072626,0.059896,0.067024,0.042553
3,err_count,22.400000,3.911521,22.000000,26.000000,23.000000,25.000000,16.000000
4,f0point5,0.660418,0.055145,0.621762,0.627907,0.705882,0.613208,0.733333
5,f1,0.689086,0.025133,0.685714,0.675000,0.676056,0.675325,0.733333
6,f2,0.725498,0.045184,0.764331,0.729730,0.648649,0.751445,0.733333
7,lift_top_group,10.896063,1.141433,9.956897,10.228572,10.105263,11.656250,12.533334
8,logloss,0.149432,0.017177,0.146212,0.166596,0.165139,0.144324,0.124889
9,max_per_class_error,0.244715,0.078337,0.172414,0.228571,0.368421,0.187500,0.266667



See the whole table with table.as_data_frame()


In [48]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_5_20221107_111813,0.943692,0.149327,0.690516,0.133869,0.208799,0.043597
StackedEnsemble_BestOfFamily_3_AutoML_5_20221107_111813,0.942005,0.15337,0.68026,0.183326,0.211714,0.0448227
StackedEnsemble_AllModels_3_AutoML_5_20221107_111813,0.941452,0.147924,0.714824,0.15417,0.206419,0.042609


In [49]:
# leader's model 

model = h2o.get_model('StackedEnsemble_BestOfFamily_4_AutoML_5_20221107_111813')

In [50]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.11577005428614927
RMSE: 0.34024998792968275
LogLoss: 0.423345278251079
Null degrees of freedom: 84
Residual degrees of freedom: 80
Null deviance: 67.15365222226669
Residual deviance: 71.96869730268344
AIC: 81.96869730268344
AUC: 0.6941031941031941
AUCPR: 0.23153790206115132
Gini: 0.3882063882063882

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0586095557863531: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,61.0,13.0,0.1757,(13.0/74.0)
1,SOZ,4.0,7.0,0.3636,(4.0/11.0)
2,Total,65.0,20.0,0.2,(17.0/85.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.058610,0.451613,19.0
1,max f2,0.058610,0.546875,19.0
2,max f0point5,0.058610,0.384615,19.0
3,max accuracy,0.186998,0.858824,0.0
4,max precision,0.058610,0.350000,19.0
5,max recall,0.009055,1.000000,72.0
6,max specificity,0.186998,0.986486,0.0
7,max absolute_mcc,0.058610,0.364542,19.0
8,max min_per_class_accuracy,0.058610,0.636364,19.0
9,max mean_per_class_accuracy,0.058610,0.730344,19.0



Gains/Lift Table: Avg response rate: 12.94 %, avg score:  4.50 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011765,0.177738,0.000000,0.000000,0.000000,0.186998,0.000000,0.186998,0.000000,0.000000,-100.000000,-100.000000,-0.013514
1,2,0.023529,0.151415,0.000000,0.000000,0.000000,0.175974,0.000000,0.181486,0.000000,0.000000,-100.000000,-100.000000,-0.027027
2,3,0.035294,0.139798,7.727273,2.575758,1.000000,0.139858,0.333333,0.167610,0.090909,0.090909,672.727273,157.575758,0.063882
3,4,0.047059,0.139274,0.000000,1.931818,0.000000,0.139743,0.250000,0.160643,0.000000,0.090909,-100.000000,93.181818,0.050369
4,5,0.058824,0.135778,0.000000,1.545455,0.000000,0.138442,0.200000,0.156203,0.000000,0.090909,-100.000000,54.545455,0.036855
5,6,0.105882,0.120668,1.931818,1.717172,0.250000,0.123283,0.222222,0.141572,0.090909,0.181818,93.181818,71.717172,0.087224
6,7,0.152941,0.083296,1.931818,1.783217,0.250000,0.102834,0.230769,0.129652,0.090909,0.272727,93.181818,78.321678,0.137592
7,8,0.200000,0.067003,3.863636,2.272727,0.500000,0.076490,0.294118,0.117144,0.181818,0.454545,286.363636,127.272727,0.292383
8,9,0.305882,0.047284,1.717172,2.080420,0.222222,0.054554,0.269231,0.095478,0.181818,0.636364,71.717172,108.041958,0.379607
9,10,0.400000,0.038707,0.000000,1.590909,0.000000,0.043321,0.205882,0.083206,0.000000,0.636364,-100.000000,59.090909,0.271499


## CLASSIFICATION PROBLEM, Pt 89
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [51]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_89_drop)
test = h2o.H2OFrame(testing_df_89)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [52]:
X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [53]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [54]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
11:29:28.925: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 54s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_6_20221107_112928

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.003569864261994912
RMSE: 0.05974834108153056
LogLoss: 0.02906197333314294
Null degrees of freedom: 1790
Residual degrees of freedom: 1780
Null deviance: 1105.7913144646327
Residual deviance: 104.09998847931801
AIC: 126.09998847931801
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8267894085661149: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1625.0,0.0,0.0,(0.0/1625.0)
1,SOZ,0.0,166.0,0.0,(0.0/166.0)
2,Total,1625.0,166.0,0.0,(0.0/1791.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.826789,1.00000,93.0
1,max f2,0.826789,1.00000,93.0
2,max f0point5,0.826789,1.00000,93.0
3,max accuracy,0.826789,1.00000,93.0
4,max precision,0.997911,1.00000,0.0
5,max recall,0.826789,1.00000,93.0
6,max specificity,0.997911,1.00000,0.0
7,max absolute_mcc,0.826789,1.00000,93.0
8,max min_per_class_accuracy,0.826789,1.00000,93.0
9,max mean_per_class_accuracy,0.826789,1.00000,93.0



Gains/Lift Table: Avg response rate:  9.27 %, avg score: 11.25 %


...


Gains/Lift Table: Avg response rate:  9.27 %, avg score:  9.29 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010050,0.953732,10.789157,10.789157,1.000000,0.976251,1.000000,0.976251,0.108434,0.108434,978.915663,978.915663,0.108434
1,2,0.020101,0.884729,10.189759,10.489458,0.944444,0.922845,0.972222,0.949548,0.102410,0.210843,918.975904,948.945783,0.210228
2,3,0.030151,0.784421,6.593373,9.190763,0.611111,0.840346,0.851852,0.913147,0.066265,0.277108,559.337349,819.076305,0.272185
3,4,0.040201,0.709179,7.792169,8.841114,0.722222,0.750177,0.819444,0.872405,0.078313,0.355422,679.216867,784.111446,0.347422
4,5,0.050251,0.610088,7.792169,8.631325,0.722222,0.656467,0.800000,0.829217,0.078313,0.433735,679.216867,763.132530,0.422658
5,6,0.100503,0.325072,5.034940,6.833133,0.466667,0.462840,0.633333,0.646029,0.253012,0.686747,403.493976,583.313253,0.646132
6,7,0.150195,0.141179,2.788209,5.494849,0.258427,0.221202,0.509294,0.505473,0.138554,0.825301,178.820902,449.484929,0.744070
7,8,0.200447,0.086292,1.318675,4.447897,0.122222,0.113786,0.412256,0.407278,0.066265,0.891566,31.867470,344.789744,0.761720
8,9,0.300391,0.033889,0.482197,3.128454,0.044693,0.052886,0.289963,0.289367,0.048193,0.939759,-51.780306,212.845434,0.704682
9,10,0.400335,0.017293,0.180824,2.392574,0.016760,0.024459,0.221757,0.223232,0.018072,0.957831,-81.917615,139.257448,0.614447




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.941792,0.015260,0.937500,0.920821,0.945799,0.963173,0.941667
1,auc,0.940936,0.020764,0.958084,0.940078,0.941775,0.957692,0.907050
2,err,0.058208,0.015260,0.062500,0.079179,0.054201,0.036827,0.058333
3,err_count,20.800000,5.118594,23.000000,27.000000,20.000000,13.000000,21.000000
4,f0point5,0.689800,0.053719,0.657895,0.633803,0.726744,0.763889,0.666667
5,f1,0.693879,0.052939,0.684932,0.666667,0.714286,0.771930,0.631579
6,f2,0.699960,0.064501,0.714286,0.703125,0.702247,0.780142,0.600000
7,lift_top_group,10.901958,1.292867,10.823529,9.216216,10.250000,12.607142,11.612904
8,logloss,0.154834,0.025071,0.140512,0.179772,0.149678,0.123418,0.180788
9,max_per_class_error,0.294834,0.076843,0.264706,0.270270,0.305556,0.214286,0.419355



See the whole table with table.as_data_frame()


In [55]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_6_20221107_112928,0.938853,0.154602,0.716903,0.176011,0.211469,0.0447192
StackedEnsemble_BestOfFamily_6_AutoML_6_20221107_112928,0.937659,0.156434,0.708963,0.184982,0.212614,0.0452048
StackedEnsemble_AllModels_1_AutoML_6_20221107_112928,0.936695,0.160283,0.69442,0.189225,0.215097,0.0462669


In [56]:
# leader's model 

model = h2o.get_model('StackedEnsemble_AllModels_4_AutoML_6_20221107_112928')

In [57]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.0532188709582946
RMSE: 0.2306921562565459
LogLoss: 0.2716438116714823
Null degrees of freedom: 169
Residual degrees of freedom: 159
Null deviance: 74.13349999626837
Residual deviance: 92.35889596830398
AIC: 114.35889596830398
AUC: 0.380952380952381
AUCPR: 0.04118620713370155
Gini: -0.23809523809523803

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.015668092950820292: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,95.0,66.0,0.4099,(66.0/161.0)
1,SOZ,5.0,4.0,0.5556,(5.0/9.0)
2,Total,100.0,70.0,0.4176,(71.0/170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.015668,0.101266,69.0
1,max f2,0.002395,0.219512,168.0
2,max f0point5,0.050556,0.072464,14.0
3,max accuracy,0.351553,0.941176,0.0
4,max precision,0.050556,0.066667,14.0
5,max recall,0.002395,1.000000,168.0
6,max specificity,0.351553,0.993789,0.0
7,max absolute_mcc,0.002505,0.367360,166.0
8,max min_per_class_accuracy,0.015668,0.444444,69.0
9,max mean_per_class_accuracy,0.015668,0.517253,69.0



Gains/Lift Table: Avg response rate:  5.29 %, avg score:  2.25 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011765,0.161128,0.000000,0.000000,0.000000,0.261080,0.000000,0.261080,0.000000,0.000000,-100.000000,-100.000000,-0.012422
1,2,0.023529,0.112460,0.000000,0.000000,0.000000,0.137199,0.000000,0.199139,0.000000,0.000000,-100.000000,-100.000000,-0.024845
2,3,0.035294,0.099648,0.000000,0.000000,0.000000,0.102219,0.000000,0.166832,0.000000,0.000000,-100.000000,-100.000000,-0.037267
3,4,0.041176,0.086499,0.000000,0.000000,0.000000,0.091696,0.000000,0.156099,0.000000,0.000000,-100.000000,-100.000000,-0.043478
4,5,0.052941,0.081565,0.000000,0.000000,0.000000,0.084105,0.000000,0.140100,0.000000,0.000000,-100.000000,-100.000000,-0.055901
5,6,0.100000,0.039658,2.361111,1.111111,0.125000,0.057982,0.058824,0.101456,0.111111,0.111111,136.111111,11.111111,0.011732
6,7,0.152941,0.028673,0.000000,0.726496,0.000000,0.034781,0.038462,0.078377,0.000000,0.111111,-100.000000,-27.350427,-0.044168
7,8,0.200000,0.024585,0.000000,0.555556,0.000000,0.026499,0.029412,0.066170,0.000000,0.111111,-100.000000,-44.444444,-0.093858
8,9,0.300000,0.019916,0.000000,0.370370,0.000000,0.021997,0.019608,0.051446,0.000000,0.111111,-100.000000,-62.962963,-0.199448
9,10,0.400000,0.015943,2.222222,0.833333,0.117647,0.018045,0.044118,0.043096,0.222222,0.333333,122.222222,-16.666667,-0.070393


## CLASSIFICATION PROBLEM, Pt 93
Predicting "Onset_channel" based on "Outcome = 11" & "Segm_type = 4", split 17 patients for training vs 1 patient for testing

In [58]:
# Parse dataframes to H2o

train = h2o.H2OFrame(training_df_93_drop)
test = h2o.H2OFrame(testing_df_93)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [59]:
X = train.columns
y = 'onset_channel'
X.remove(y)
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
y

'onset_channel'

In [60]:
# Import H2o AutoML

from h2o.automl import H2OAutoML

In [61]:
# Run H2o AutoML for 600 secs = 10 minutes

aml = H2OAutoML(max_runtime_secs=600,   
                balance_classes=True,
                stopping_metric='AUC',
                seed=121) 


%time aml.train(x=X, y=y, training_frame = train)

AutoML progress: |
11:40:43.430: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Wall time: 9min 56s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_7_20221107_114043

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0046655467146221805
RMSE: 0.06830480740491243
LogLoss: 0.03134177648331486
Null degrees of freedom: 1800
Residual degrees of freedom: 1796
Null deviance: 1121.3960197292538
Residual deviance: 112.89307889290008
AIC: 122.89307889290008
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7640017728557841: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,1632.0,0.0,0.0,(0.0/1632.0)
1,SOZ,0.0,169.0,0.0,(0.0/169.0)
2,Total,1632.0,169.0,0.0,(0.0/1801.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.764002,1.00000,83.0
1,max f2,0.764002,1.00000,83.0
2,max f0point5,0.764002,1.00000,83.0
3,max accuracy,0.764002,1.00000,83.0
4,max precision,0.999690,1.00000,0.0
5,max recall,0.764002,1.00000,83.0
6,max specificity,0.999690,1.00000,0.0
7,max absolute_mcc,0.764002,1.00000,83.0
8,max min_per_class_accuracy,0.764002,1.00000,83.0
9,max mean_per_class_accuracy,0.764002,1.00000,83.0



Gains/Lift Table: Avg response rate:  9.38 %, avg score: 11.46 %


...


Gains/Lift Table: Avg response rate:  9.38 %, avg score:  9.38 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010550,0.951541,10.656805,10.656805,1.000000,0.973684,1.000000,0.973684,0.112426,0.112426,965.680473,965.680473,0.112426
1,2,0.020544,0.870152,10.656805,10.656805,1.000000,0.915013,1.000000,0.945141,0.106509,0.218935,965.680473,965.680473,0.218935
2,3,0.030539,0.786438,7.104536,9.494244,0.666667,0.819803,0.890909,0.904122,0.071006,0.289941,610.453649,849.424422,0.286264
3,4,0.040533,0.657235,7.696581,9.050985,0.722222,0.725418,0.849315,0.860058,0.076923,0.366864,669.658120,805.098484,0.360124
4,5,0.050527,0.611381,5.328402,8.314650,0.500000,0.633215,0.780220,0.815188,0.053254,0.420118,432.840237,731.464985,0.407863
5,6,0.100500,0.312986,5.091584,6.712021,0.477778,0.445577,0.629834,0.631403,0.254438,0.674556,409.158448,571.202066,0.633502
6,7,0.150472,0.168389,2.368179,5.269416,0.222222,0.227887,0.494465,0.497394,0.118343,0.792899,136.817883,426.941636,0.708953
7,8,0.200444,0.101235,1.420907,4.309954,0.133333,0.130705,0.404432,0.405976,0.071006,0.863905,42.090730,330.995427,0.732165
8,9,0.300389,0.044397,0.828863,3.151735,0.077778,0.066421,0.295749,0.293000,0.082840,0.946746,-17.113741,215.173523,0.713290
9,10,0.400333,0.020813,0.177613,2.409236,0.016667,0.030095,0.226075,0.227365,0.017751,0.964497,-82.238659,140.923602,0.622585




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.940809,0.011310,0.926630,0.947826,0.932796,0.954802,0.941989
1,auc,0.943222,0.013837,0.940800,0.931379,0.951798,0.962346,0.929788
2,err,0.059191,0.011310,0.073370,0.052174,0.067204,0.045198,0.058011
3,err_count,21.400000,4.615192,27.000000,18.000000,25.000000,16.000000,21.000000
4,f0point5,0.687694,0.081039,0.552486,0.762712,0.685714,0.733333,0.704225
5,f1,0.678724,0.062682,0.597015,0.750000,0.657534,0.733333,0.655738
6,f2,0.673093,0.058400,0.649351,0.737705,0.631579,0.733333,0.613497
7,lift_top_group,10.799900,1.445967,12.689655,9.324325,9.538462,11.800000,10.647058
8,logloss,0.156196,0.020834,0.153836,0.177547,0.164106,0.122127,0.163366
9,max_per_class_error,0.328732,0.066380,0.310345,0.270270,0.384615,0.266667,0.411765



See the whole table with table.as_data_frame()


In [62]:
# H2o AutoML leaderboard 

lb = aml.leaderboard
lb.head(rows=3)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_4_AutoML_7_20221107_114043,0.941911,0.156182,0.714947,0.186102,0.214454,0.0459905
StackedEnsemble_AllModels_3_AutoML_7_20221107_114043,0.940633,0.157253,0.715011,0.18549,0.214417,0.0459746
StackedEnsemble_AllModels_4_AutoML_7_20221107_114043,0.937825,0.15855,0.717832,0.180185,0.213561,0.0456084


In [63]:
# leader's model 

model = h2o.get_model('StackedEnsemble_BestOfFamily_4_AutoML_7_20221107_114043')

In [64]:
# Use Leader's model on test set

model.model_performance(test)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.03447699558075396
RMSE: 0.18567982006872466
LogLoss: 0.1744305966218222
Null degrees of freedom: 159
Residual degrees of freedom: 155
Null deviance: 58.74341293582078
Residual deviance: 55.81779091898306
AIC: 65.81779091898306
AUC: 0.6461038961038961
AUCPR: 0.1829826463688203
Gini: 0.29220779220779214

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1587562119059918: 


,,NON_SOZ,SOZ,Error,Rate
0,NON_SOZ,149.0,5.0,0.0325,(5.0/154.0)
1,SOZ,3.0,3.0,0.5,(3.0/6.0)
2,Total,152.0,8.0,0.05,(8.0/160.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.158756,0.428571,7.0
1,max f2,0.158756,0.468750,7.0
2,max f0point5,0.158756,0.394737,7.0
3,max accuracy,0.398989,0.962500,1.0
4,max precision,0.398989,0.500000,1.0
5,max recall,0.000438,1.000000,156.0
6,max specificity,0.540916,0.993506,0.0
7,max absolute_mcc,0.158756,0.407550,7.0
8,max min_per_class_accuracy,0.024649,0.571429,69.0
9,max mean_per_class_accuracy,0.158756,0.733766,7.0



Gains/Lift Table: Avg response rate:  3.75 %, avg score:  4.24 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01250,0.388906,13.333333,13.333333,0.5000,0.469953,0.500000,0.469953,0.166667,0.166667,1233.333333,1233.333333,0.160173
1,2,0.02500,0.350219,0.000000,6.666667,0.0000,0.374644,0.250000,0.422298,0.000000,0.166667,-100.000000,566.666667,0.147186
2,3,0.03125,0.271507,26.666667,10.666667,1.0000,0.272004,0.400000,0.392239,0.166667,0.333333,2566.666667,966.666667,0.313853
3,4,0.04375,0.166630,0.000000,7.619048,0.0000,0.221209,0.285714,0.343373,0.000000,0.333333,-100.000000,661.904762,0.300866
4,5,0.05000,0.156458,26.666667,10.000000,1.0000,0.158756,0.375000,0.320296,0.166667,0.500000,2566.666667,900.000000,0.467532
5,6,0.10000,0.100117,0.000000,5.000000,0.0000,0.126774,0.187500,0.223535,0.000000,0.500000,-100.000000,400.000000,0.415584
6,7,0.15000,0.076039,0.000000,3.333333,0.0000,0.085675,0.125000,0.177582,0.000000,0.500000,-100.000000,233.333333,0.363636
7,8,0.20000,0.058647,0.000000,2.500000,0.0000,0.069310,0.093750,0.150514,0.000000,0.500000,-100.000000,150.000000,0.311688
8,9,0.30000,0.040754,0.000000,1.666667,0.0000,0.047842,0.062500,0.116290,0.000000,0.500000,-100.000000,66.666667,0.207792
9,10,0.40000,0.029550,0.000000,1.250000,0.0000,0.036034,0.046875,0.096226,0.000000,0.500000,-100.000000,25.000000,0.103896
